In [64]:
from dotenv import load_dotenv
import openai
import os
from openai import AzureOpenAI

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
OPENAI_GPT4_DEPLOYMENT_NAME = os.getenv("OPENAI_GPT4_DEPLOYMENT_NAME")
 
AZURE_COMPUTER_VISION_ENDPOINT = os.getenv("AZURE_COMPUTER_VISION_ENDPOINT")
AZURE_COMPUTER_VISION_KEY = os.getenv("AZURE_COMPUTER_VISION_KEY")


In [55]:
import os
import json
from azure.ai.vision.imageanalysis import ImageAnalysisClient
from azure.ai.vision.imageanalysis.models import VisualFeatures
from azure.core.credentials import AzureKeyCredential

def analyze_image(image_filename):
    client = ImageAnalysisClient(
        endpoint=AZURE_COMPUTER_VISION_ENDPOINT,
        credential=AzureKeyCredential(AZURE_COMPUTER_VISION_KEY)
    )

    with open(image_filename, "rb") as image_stream:
        result = client.analyze(
            image_stream,
            visual_features=[VisualFeatures.CAPTION, VisualFeatures.READ, VisualFeatures.DENSE_CAPTIONS, VisualFeatures.OBJECTS, VisualFeatures.PEOPLE, VisualFeatures.TAGS],
            gender_neutral_caption=True,  # Optional (default is False)
        )

    print("Image analysis results:")
    # Print caption results to the console
    print(" Caption:")
    if result.caption is not None:
        print(f"   '{result.caption.text}', Confidence {result.caption.confidence:.4f}")

    result_dict = result.as_dict()
    result_json = json.dumps(result_dict)
    return result_json
            


#### Analyze a picture using Azure Cognitve services to extract text from a picture

In [56]:
image_url = "./data/retail/bike.jpg"
json_result = analyze_image(image_url)

Image analysis results:
 Caption:
   'a blue and pink polo shirt', Confidence 0.7036


#### Extract all dense captions from the json result

In [57]:
import json
dict = json.loads(json_result)
dense_captions = dict["denseCaptionsResult"]["values"]


#### Generate a product description from the text extracted from the photo using OpenAI

In [61]:
client = AzureOpenAI(
  azure_endpoint = OPENAI_DEPLOYMENT_ENDPOINT, 
  api_key=OPENAI_API_KEY,  
  api_version="2024-02-15-preview"
)


In [65]:
def call_openAI(text):
    message_text = [
    {"role":"system","content":"You are an AI assistant that generates product descriptions for an online retail website especializing in clothing. Given the user's input, generate a Product Description."},
    {"role":"user","content":text}]

    response = client.chat.completions.create(
        model=OPENAI_GPT4_DEPLOYMENT_NAME,
        messages = message_text,
        temperature=0.7,
        max_tokens=800,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )

    return response.choices[0].message.content

In [66]:
count = 1
for caption in dense_captions:
    result = call_openAI(caption["text"])
    print("Option " + str(count) + ": " + result)
    count = count + 1


Option 1: **Title:** Men's Dual-Tone Blue and Pink Cotton Polo Shirt

**Product Description:**

Elevate your casual wardrobe with the vibrant charm of our Men's Dual-Tone Blue and Pink Cotton Polo Shirt. This stylish polo combines classic comfort with a modern twist, perfect for those who appreciate a pop of color in their everyday attire.

**Key Features:**

- **Premium Fabric:** Crafted from high-quality, breathable cotton fabric that offers a soft touch against your skin while ensuring long-lasting wear.
- **Striking Dual-Tone Design:** The shirt features a bold blue base complemented by eye-catching pink accents, creating a harmonious blend of colors that will stand out in any setting.
- **Classic Polo Collar:** A neat, ribbed collar with a two-button placket provides a timeless polo look, blending effortlessly with both casual and semi-formal ensembles.
- **Short Sleeves:** The short sleeve design keeps you cool and comfortable, making it ideal for warmer days or as a layering pie